In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import pyspark.sql.functions as psf
from math import sqrt

In [3]:
# Build a SparkSession; SparkSession provides a single
spark = SparkSession\
        .builder\
        .appName("graph")\
        .getOrCreate()

In [4]:
df = spark.read.option("header", True).csv("new_data.csv")
df = df.select("iid","pid","match","wave")
df = df.toPandas()

result = df[df.isnull().values==True]
print(result)

#df = df.drop(df.index[[1756,1766,1776,1786,1796,1806,1816,1826,1836,1846]])
# result = df[df.isnull().values==True]
# print(result)

Empty DataFrame
Columns: [iid, pid, match, wave]
Index: []


In [5]:
df["iid"] = df['iid'].astype(int)
df["pid"] = df['pid'].astype(int)
# df["iid"] = df["iid"] - 1
# df["pid"] = df["pid"] - 1

In [6]:
df.head(10)

,iid,pid,match,wave
0,0,10,0,1
1,0,11,0,1
2,0,12,1,1
3,0,13,1,1
4,0,14,1,1
5,0,15,0,1
6,0,16,0,1
7,0,17,0,1
8,0,18,1,1
9,0,19,0,1


In [7]:
sc = spark.sparkContext
sqlContest = SQLContext(sc)
df = sqlContest.createDataFrame(df)

In [8]:
node = df.select("iid","wave")
node = node.toPandas()
node = node.drop_duplicates("iid", 'first')
node = node.sort_values(by='iid')
node

,iid,wave
0,0,1
10,1,1
20,2,1
556,3,1
566,4,1
576,5,1
586,6,1
596,7,1
605,8,1
615,9,1


In [9]:
nodefile="/home/mf3200/Bigdataproject/node_new0.csv"
node.to_csv(path_or_buf=nodefile, header = False, sep=",")

In [10]:
nodefile = spark.read.csv("/home/mf3200/Bigdataproject/node_new0.csv")
node  = nodefile.toPandas()
node

,_c0,_c1,_c2
0,0,0,1
1,10,1,1
2,20,2,1
3,556,3,1
4,566,4,1
5,576,5,1
6,586,6,1
7,596,7,1
8,605,8,1
9,615,9,1


In [11]:
nodefile="/home/mf3200/Bigdataproject/node_new1.csv"
node.to_csv(path_or_buf=nodefile, header = False, sep=",")

In [12]:
edge = df.select("iid","pid","match","wave")
edge = edge.toPandas()
edge

,iid,pid,match,wave
0,0,10,0,1
1,0,11,0,1
2,0,12,1,1
3,0,13,1,1
4,0,14,1,1
5,0,15,0,1
6,0,16,0,1
7,0,17,0,1
8,0,18,1,1
9,0,19,0,1


In [13]:
edgefile="/home/mf3200/Bigdataproject/edge_new0.csv"
edge.to_csv(path_or_buf=edgefile, header = False, sep=",")

In [14]:
edgefile = spark.read.csv("/home/mf3200/Bigdataproject/edge_new0.csv")
edge  = edgefile.toPandas()
edge

,_c0,_c1,_c2,_c3,_c4
0,0,0,10,0,1
1,1,0,11,0,1
2,2,0,12,1,1
3,3,0,13,1,1
4,4,0,14,1,1
5,5,0,15,0,1
6,6,0,16,0,1
7,7,0,17,0,1
8,8,0,18,1,1
9,9,0,19,0,1


In [15]:
edgefile="/home/mf3200/Bigdataproject/edge_new1.csv"
edge.to_csv(path_or_buf=edgefile, header = False, sep=",")